In [1]:
import pandas as pd

# Display settings to show all columns and improve readability
pd.set_option('display.max_columns', None)        # Remove column limit
pd.set_option('display.max_rows', 50)             # Show up to 50 rows
pd.set_option('display.expand_frame_repr', False) # Prevent line wrapping

# 1. Define the file path
path = r'C:\Users\MartínPazYáñez\Desktop\iron\bootcamp\TFM\steam_games_dataset\games_normalized_forAPI_call.csv'

# 2. Load the CSV file into a DataFrame
df_steam_normalized= pd.read_csv(path)

# 3. Show the first rows of the DataFrame
df_steam_normalized = df_steam_normalized.rename(columns={'Unnamed: 0': 'app_id'})

df_steam_normalized.head()

,app_id,name,name_normalized,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,achievements,recommendations,notes,supported_languages,full_audio_languages,packages,developers,publishers,categories,genres,screenshots,movies,user_score,score_rank,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,total_ratings
0,578080,PUBG: BATTLEGROUNDS,PUBG BATTLEGROUNDS,2017-12-21,13,0.00,0,LAND Drop into an ever-growing and changin...,LAND Drop into an ever-growing and changin...,"PUBG: BATTLEGROUNDS, the high-stakes winner-ta...",NaN,https://shared.akamai.steamstatic.com/store_it...,https://www.pubg.com,https://support.pubg.com/hc/en-us,NaN,True,False,False,0,NaN,37,1753261,NaN,"['English', 'Korean', 'Simplified Chinese', 'F...",[],[],['PUBG Corporation'],"['KRAFTON, Inc.']","['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...",['https://shared.akamai.steamstatic.com/store_...,[],0,NaN,1520457,1037487,100000000 - 200000000,23787,730,6082,302,0,314682,"{'Survival': 14893, 'Shooter': 12788, 'Battle ...",2557944
1,252490,Rust,Rust,2018-02-08,0,19.99,5,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,"“Rust is one of the cruelest games on Steam, a...",https://shared.akamai.steamstatic.com/store_it...,http://rust.facepunch.com/,http://support.facepunchstudios.com/,NaN,True,True,False,69,https://www.metacritic.com/game/pc/rust?ftag=M...,92,1081623,Contains violence and caveman themed nudity. I...,"['English', 'French', 'Italian', 'German', 'Sp...","['English', 'French', 'Italian', 'German', 'Sp...","[{'title': 'Buy Rust', 'description': '', 'sub...",['Facepunch Studios'],['Facepunch Studios'],"['Multi-player', 'MMO', 'PvP', 'Online PvP', '...","['Action', 'Adventure', 'Indie', 'Massively Mu...",['https://shared.akamai.steamstatic.com/store_...,[],0,NaN,1071135,156649,20000000 - 50000000,24264,1246,3044,373,50,143870,"{'Survival': 18782, 'Crafting': 11939, 'Multip...",1227784
2,2358720,Black Myth: Wukong,Black Myth Wukong,2024-08-19,13,59.99,2,Digital Deluxe Edition The Black Myth: Wukong ...,Black Myth: Wukong is an action RPG rooted in ...,Black Myth: Wukong is an action RPG rooted in ...,NaN,https://shared.akamai.steamstatic.com/store_it...,https://www.gamesci.cn/wukong,NaN,cs@gamesci.com.cn,True,False,False,0,NaN,81,863021,NaN,"['English', 'French', 'German', 'Spanish - Spa...","['English', 'Simplified Chinese', 'Czech']","[{'title': 'Buy Black Myth: Wukong', 'descript...",['Game Science'],['Game Science'],"['Single-player', 'Steam Achievements', 'Full ...","['Action', 'Adventure', 'RPG']",['https://shared.akamai.steamstatic.com/store_...,[],0,NaN,1111720,38378,50000000 - 100000000,3272,399,2812,135,0,15004,"{'Mythology': 9480, 'Action RPG': 7769, 'Actio...",1150098
3,1245620,ELDEN RING,ELDEN RING,2022-02-24,16,38.99,2,ELDEN RING Shadow of the Erdtree Edition ELDEN...,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG Ri...,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,“Put a ring on it.” 10/10 – IGN “An unmissable...,https://shared.akamai.steamstatic.com/store_it...,NaN,https://www.bandainamcoent.com/support,NaN,True,False,False,94,https://www.metacritic.com/game/pc/elden-ring?...,42,796256,NaN,"['English', 'French', 'Italian', 'German', 'Sp...","['English', 'Arabic']","[{'title': 'Buy ELDEN RING', 'description': ''...","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...","['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'RPG']",['https://shared.akamai.steamstatic.com/store_...,[],0,NaN,981540,75137,20000000 - 50000000,8155,500,5832,224,35,29529,"{'Souls-like': 6994, 'Open World': 5078, 'Dark...",1056677
4,553

In [ ]:
import pandas as pd
import requests
import time

# Your API key
API_KEY = "47f69e9ce1404f178e207f1f826854d8"

# Try to load previous work, or start fresh
try:
    df_result = pd.read_csv("steam_enriched.csv")
    start = len(df_result)
    print(f"✓ Continuing from row {start}")
except:
    df_result = pd.DataFrame()
    start = 0
    print("✓ Starting from the beginning")

# Process each game
total = len(df_steam_normalized)
print(f"\nProcessing {total - start} games")
print(f"Estimated time: ~{((total - start) * 1.5) / 3600:.1f} hours\n")

# Store results in a list (faster than concat each time)
results_list = []

for i in range(start, total):
    # Get game name
    game_name = df_steam_normalized.iloc[i]["name_normalized"]
    
    print(f"[{i+1}/{total}] {game_name}", end=" ")
    
    # Search in RAWG
    try:
        url = f"https://api.rawg.io/api/games?key={API_KEY}&search={game_name}&page_size=1"
        response = requests.get(url, timeout=20)
        data = response.json()
    except:
        data = {}
    
    # Get Steam data as dict
    steam_data = df_steam_normalized.iloc[i].to_dict()
    
    # Add RAWG data if found
    if data.get("results"):
        rawg_data = pd.json_normalize(data["results"][0]).add_prefix("rawg_").iloc[0].to_dict()
        combined = {**steam_data, **rawg_data, "rawg_status": "FOUND"}
        print("OK")
    else:
        combined = {**steam_data, "rawg_status": "NOT_FOUND"}
        print("KO")
    
    # Add to list
    results_list.append(combined)
    
    # Save every 100
    if (i + 1) % 100 == 0:
        # Convert list to DataFrame and append
        df_batch = pd.DataFrame(results_list)
        df_result = pd.concat([df_result, df_batch], ignore_index=True)
        df_result.to_csv("steam_enriched.csv", index=False)
        
        # Clear list
        results_list = []
        
        print(f"\n SAVED ({i+1}/{total} rows - {((i+1)/total)*100:.1f}%)\n")
    
    # Pause
    time.sleep(1.5)

# Save remaining
if results_list:
    df_batch = pd.DataFrame(results_list)
    df_result = pd.concat([df_result, df_batch], ignore_index=True)
    df_result.to_csv("steam_enriched.csv", index=False)

print(f"\n DONE {len(df_result)} games processed")

✓ Continuing from row 2500

Processing 17500 games
Estimated time: ~7.3 hours

[2501/20000] Fluffy Store ✓
[2502/20000] Romance of the Three Kingdoms XI with Power Up Kit ✓
[2503/20000] Cataclismo ✓
[2504/20000] Millennia ✓
[2505/20000] Heretic's Fork ✓
[2506/20000] The Backrooms Game FREE Edition ✓
[2507/20000] ANTONBLAST ✓
[2508/20000] Killer Klowns from Outer Space The Game ✓
[2509/20000] Ballads of Hongye REBORN ✓
[2510/20000] Riichi City × The Quintessential Quintuplets Specials 2 ✓
[2511/20000] Atlas Fallen Reign Of Sand ✓
[2512/20000] Coloring Game ✓
[2513/20000] Ashen ✓
[2514/20000] 风信楼 ✓
[2515/20000] Lost in Vivo ✓
[2516/20000] MaitetsuPure Station ✓
[2517/20000] Survivalist Invisible Strain ✓
[2518/20000] Island Saver ✓
[2519/20000] DOKA 2 KISHKI EDITION ✓
[2520/20000] Disciples Liberation ✓
[2521/20000] Atelier Ryza 2 Lost Legends & the Secret Fairy ✓
[2522/20000] Serious Sam Classics Revolution ✓
[2523/20000] Alone in the Dark ✓
[2524/20000] SCP Escape Together ✓
[2525/2000

C:\Users\MartínPazYáñez\AppData\Local\Temp\ipykernel_2668\472560183.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result = pd.concat([df_result, df_batch], ignore_index=True)



💾 SAVED! (13800/20000 rows - 69.0%)

[13801/20000] Werewolf The Apocalypse - The Book of Hungry Names ✓
[13802/20000] TYPECAST ✓
[13803/20000] Food Delivery Simulator ✓
[13804/20000] Illyriad - Grand Strategy MMO ✓
[13805/20000] Dingletopia Nation Under Siege (by Orcs) ✓
[13806/20000] Ne no Kami The Two Princess Knights of Kyoto ✓
[13807/20000] Magenta Horizon - Neverending Harvest ✓
[13808/20000] Bombernauts ✓
[13809/20000] Harvest Days My Dream Farm ✓
[13810/20000] Without Romance ✓
[13811/20000] Next Up Hero ✓
[13812/20000] Outpost ✓
[13813/20000] Jar Wars ✓
[13814/20000] Try To Fall Asleep ✓
[13815/20000] Is this Game Trying to Kill Me? Preface ✓
[13816/20000] Russian roulette ✓
[13817/20000] HereSphere VR Video Player ✓
[13818/20000] Red Haze ✓
[13819/20000] Witch and Council The Card ✓
[13820/20000] Pipe Push Paradise ✓
[13821/20000] Music Wars Empire ✓
[13822/20000] Dofamine ✓
[13823/20000] Mr. Fast ✓
[13824/20000] Microsoft Allegiance ✓
[13825/20000] Nancy Drew The Secret of S

C:\Users\MartínPazYáñez\AppData\Local\Temp\ipykernel_2668\472560183.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result = pd.concat([df_result, df_batch], ignore_index=True)



💾 SAVED! (14400/20000 rows - 72.0%)

[14401/20000] Hurricane ✓
[14402/20000] Octave ✓
[14403/20000] Dance of Death Du Lac & Fey ✓
[14404/20000] Zomborg ✓
[14405/20000] Angvik 2 ✓
[14406/20000] Castle on the Coast ✓
[14407/20000] Daily Chthonicle Editor's Edition ✓
[14408/20000] Rogue Spirit ✓
[14409/20000] Discovery Tour by Assassin’s Creed Ancient Egypt ✓
[14410/20000] 風色幻想5赤月戰爭 ✓
[14411/20000] Warriors' Wrath ✓
[14412/20000] The Girl's Moving Castle ✓
[14413/20000] E-Startup ✓
[14414/20000] Fly Punch Boom! ✓
[14415/20000] Succubus Girl Story ✓
[14416/20000] Rituals in the Dark ✓
[14417/20000] Moonlight Pulse ✓
[14418/20000] Creatures Docking Station ✓
[14419/20000] Waifus Smash ✓
[14420/20000] Sentimental Death Loop ✓
[14421/20000] Dread Weight ✓
[14422/20000] My Protogen Engineer ⚙️ ✓
[14423/20000] Presidents Tower Defense ✓
[14424/20000] RPG Maker VX ✓
[14425/20000] River City Rival Showdown ✓
[14426/20000] Toy Tinker Simulator Prologue ✓
[14427/20000] Odysseus Kosmos and his Robo

...


💾 SAVED! (16900/20000 rows - 84.5%)

[16901/20000] Noise Hunters ✓
[16902/20000] Rexodus A VR Story Experience ✓
[16903/20000] Achievement Lurker Easiest Cosmetic Numbers ✓
[16904/20000] Milanoir ✓
[16905/20000] MegaFactory Titan ✓
[16906/20000] Estencel ✓
[16907/20000] FORECLOSED ✓
[16908/20000] Thunder Helix ✓
[16909/20000] Higurashi When They Cry Hou+ ✓
[16910/20000] Recon Control ✓
[16911/20000] Sir Lovelot ✓
[16912/20000] Chaos Faction Legacy Collection ✓
[16913/20000] Last Labyrinth ✓
[16914/20000] Anime City ✓
[16915/20000] 舔狗模拟器 ✓
[16916/20000] Rebel! Pure love fighters! ✓
[16917/20000] Persian Nights 2 The Moonlight Veil ✓
[16918/20000] What's Your Gender? ✓
[16919/20000] The Repairing Mantis ✓
[16920/20000] I commissioned some cats ✓
[16921/20000] Guess who ? ✓
[16922/20000] 3D Organon ✓
[16923/20000] Smart City Plan ✓
[16924/20000] Chocolate makes you happy 3 ✓
[16925/20000] Anima Flux ✓
[16926/20000] TitTok ✓
[16927/20000] Psychiatrist Simulator ✓
[16928/20000] Survivor ✓


C:\Users\MartínPazYáñez\AppData\Local\Temp\ipykernel_2668\472560183.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result = pd.concat([df_result, df_batch], ignore_index=True)



💾 SAVED! (18700/20000 rows - 93.5%)

[18701/20000] Silmaris Dice Kingdom ✓
[18702/20000] The Legacy Forgotten Gates Collector's Edition ✓
[18703/20000] Lazaretto ✓
[18704/20000] Magic Potion Explorer ✓
[18705/20000] 位面穿越者-三国传(A Bit Crosser-Three Kingdoms) ✓
[18706/20000] Knight's Retreat ✓
[18707/20000] Police Response VR Disturbance ✓
[18708/20000] Achievement Simulator ✓
[18709/20000] Camp Wars ✓
[18710/20000] Space Bob vs. The Replicons ✓
[18711/20000] WAIFU IMPACT ✓
[18712/20000] Burned Land ✓
[18713/20000] AFTERGRINDER ✓
[18714/20000] Natsuno-Kanata Beyond Summer ✓
[18715/20000] Time Drifter ✓
[18716/20000] The Monarch ✓
[18717/20000] I'm Surrounded by Classical Beauties! ✓
[18718/20000] Meiji Tokyo Renka Full Moon ✓
[18719/20000] Rough Justice '84 ✓
[18720/20000] Dance of Cards ✓
[18721/20000] Hunger Apartment （蚀狱） ✓
[18722/20000] Gopnik Simulator ✓
[18723/20000] 1001 Hugs ✓
[18724/20000] After The End The Harvest ✓
[18725/20000] aerofly RC 10 - RC Flight Simulator ✓
[18726/2000

C:\Users\MartínPazYáñez\AppData\Local\Temp\ipykernel_2668\472560183.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result = pd.concat([df_result, df_batch], ignore_index=True)



💾 SAVED! (19900/20000 rows - 99.5%)

[19901/20000] Tiny Witch ✓
[19902/20000] The Monster Within ✓
[19903/20000] Geology Business ✓
[19904/20000] VRMultigames ✓
[19905/20000] The Bradwell Conspiracy ✓
[19906/20000] Russian Soul Simulator ✓
[19907/20000] Just Ignore Them Brea's Story Tape 1 ✓
[19908/20000] Hand In Hand ✓
[19909/20000] Monster Garden ✓
[19910/20000] Miner Lou ✓
[19911/20000] Lost Lands The Wanderer Collector's Edition ✓
[19912/20000] Tiny Dangerous Dungeons ✓
[19913/20000] Football School ✓
[19914/20000] Staden under Gamlestaden ✓
[19915/20000] Horror Fish Simulator ✓
[19916/20000] 10mg ) ✓
[19917/20000] Instinct ✓
[19918/20000] Connection The Nightmare Within ✓
[19919/20000] Cute Lady ✓
[19920/20000] Gift of Parthax ✓
[19921/20000] The Dragonhood ✓
[19922/20000] DIY MY LADY IN VR WORLD ✓
[19923/20000] We Slay Monsters ✓
[19924/20000] Traffic Jams ✓
[19925/20000] Gwen The Hen 64 ✓
[19926/20000] Haxity ✓
[19927/20000] Cobra Kai 2 Dojos Rising ✓
[19928/20000] Bunny Bounce

In [10]:
import pandas as pd

# Load the dataset
df = pd.read_csv("steam_enriched.csv")

print(f"Total before: {len(df)} games")

# Keep ONLY rows with FOUND (remove NOT_FOUND, ERROR, empty, etc.)
df_clean = df[df["rawg_status"] == "FOUND"]

print(f"Total after: {len(df_clean)} games")
print(f"Removed: {len(df) - len(df_clean)} games")

# Check which values existed in rawg_status
print("\nStatus distribution:")
print(df["rawg_status"].value_counts())

# Save the cleaned dataset
df_clean.to_csv("steam_enriched_clean.csv", index=False)

print("\n✅ Clean file saved: steam_enriched_clean.csv")


Total before: 20000 games
Total after: 19901 games
Removed: 99 games

Status distribution:
rawg_status
FOUND        19901
NOT_FOUND       99
Name: count, dtype: int64

✅ Clean file saved: steam_enriched_clean.csv
